In [97]:
import pandas as pd
import pickle 
import numpy as np


# READ DATA

In [98]:
with open('data/transform/df_participant_selections_match.pkl', 'rb') as file:
    participants_selections_match = pickle.load(file)

In [99]:
match_data = pd.read_csv('data/transform/match_team.csv',sep=';')

In [100]:
with open('data/transform/model_names_participants.pkl', 'rb') as file:
    model_names_participants = pickle.load(file)

In [101]:
model_names_participants

['teamId',
 'champExperience',
 'Position_BOTTOM',
 'Position_JUNGLE',
 'Position_MIDDLE',
 'Position_TOP',
 'Position_UTILITY',
 'championBan_-1',
 'championBan_1',
 'championBan_2',
 'championBan_3',
 'championBan_4',
 'championBan_5',
 'championBan_6',
 'championBan_7',
 'championBan_8',
 'championBan_9',
 'championBan_10',
 'championBan_11',
 'championBan_12',
 'championBan_13',
 'championBan_14',
 'championBan_15',
 'championBan_16',
 'championBan_17',
 'championBan_18',
 'championBan_19',
 'championBan_20',
 'championBan_21',
 'championBan_22',
 'championBan_23',
 'championBan_24',
 'championBan_25',
 'championBan_26',
 'championBan_27',
 'championBan_28',
 'championBan_29',
 'championBan_30',
 'championBan_31',
 'championBan_32',
 'championBan_33',
 'championBan_34',
 'championBan_35',
 'championBan_36',
 'championBan_37',
 'championBan_38',
 'championBan_39',
 'championBan_40',
 'championBan_41',
 'championBan_42',
 'championBan_43',
 'championBan_44',
 'championBan_45',
 'cham

In [102]:
participants_selections_match

[array([[1, 13182, False, ..., False, False, False],
        [1, 12374, False, ..., False, False, False],
        [1, 13929, False, ..., False, False, False],
        ...,
        [0, 12944, False, ..., False, False, False],
        [0, 12398, True, ..., False, False, False],
        [0, 9321, False, ..., False, False, False]], dtype=object),
 array([[1, 13182, False, ..., False, False, False],
        [1, 12374, False, ..., False, False, False],
        [1, 13929, False, ..., False, False, False],
        ...,
        [0, 12944, False, ..., False, False, False],
        [0, 12398, True, ..., False, False, False],
        [0, 9321, False, ..., False, False, False]], dtype=object),
 array([[1, 13182, False, ..., False, False, False],
        [1, 12374, False, ..., False, False, False],
        [1, 13929, False, ..., False, False, False],
        ...,
        [0, 12944, False, ..., False, False, False],
        [0, 12398, True, ..., False, False, False],
        [0, 9321, False, ..., Fal

In [103]:
match_data=match_data.drop(['matchId','gameEndedInSurrender_1','gameEndedInSurrender_2'],axis=1)
match_data

,baronKills_1,championKills_1,dragonKills_1,inhibitorKills_1,riftHeraldKills_1,towerKills_1,baronKills_2,championKills_2,dragonKills_2,inhibitorKills_2,riftHeraldKills_2,towerKills_2,win_1
0,1,18,0,1,1,6,0,35,4,2,1,9,0
1,1,48,2,2,1,9,1,45,2,0,1,5,1
2,0,17,2,0,0,2,1,36,1,1,2,9,0
3,0,19,1,0,1,3,0,6,0,0,0,0,1
4,0,21,0,0,2,3,1,32,3,2,0,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142768,0,17,1,0,1,5,2,37,3,1,1,9,0
142769,1,37,4,1,2,8,0,10,0,0,0,2,1
142770,0,40,4,2,1,11,1,20,0,0,0,2,1
142771,1,41,3,0,0,5,1,50,2,1,2,9,0


In [104]:
from sklearn.model_selection import train_test_split


In [105]:
X_pandas = match_data.drop(columns='win_1')
X_matrix = np.array(participants_selections_match).astype('float64')
y = match_data['win_1']

In [106]:
X_pandas_train, X_pandas_test, X_matrices_train, X_matrices_test,y_train,y_test = train_test_split(X_pandas, X_matrix,y, test_size=0.2, random_state=42)

In [72]:
X_pandas_train

,baronKills_1,championKills_1,dragonKills_1,inhibitorKills_1,riftHeraldKills_1,towerKills_1,baronKills_2,championKills_2,dragonKills_2,inhibitorKills_2,riftHeraldKills_2,towerKills_2
81549,0,19,1,0,1,4,0,6,1,0,0,1
57917,0,46,2,1,2,9,1,48,3,2,0,8
45948,0,27,0,0,0,2,1,37,3,2,2,10
28596,0,34,2,2,2,9,0,13,1,0,0,2
105557,0,26,1,0,0,1,1,33,2,2,2,10
...,...,...,...,...,...,...,...,...,...,...,...,...
110268,3,25,2,2,1,8,0,47,4,2,1,10
119879,0,24,1,0,1,3,2,39,2,1,1,7
103694,0,4,0,0,1,0,0,21,2,0,1,3
131932,1,31,2,0,1,4,1,35,2,1,0,10


In [73]:
X_matrices_train

array([[[1.0000e+00, 1.3182e+04, 0.0000e+00, ..., 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.0000e+00, 1.2374e+04, 0.0000e+00, ..., 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.0000e+00, 1.3929e+04, 0.0000e+00, ..., 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        ...,
        [0.0000e+00, 1.2944e+04, 0.0000e+00, ..., 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.2398e+04, 1.0000e+00, ..., 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [0.0000e+00, 9.3210e+03, 0.0000e+00, ..., 0.0000e+00,
         0.0000e+00, 0.0000e+00]],

       [[1.0000e+00, 1.3182e+04, 0.0000e+00, ..., 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.0000e+00, 1.2374e+04, 0.0000e+00, ..., 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [1.0000e+00, 1.3929e+04, 0.0000e+00, ..., 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        ...,
        [0.0000e+00, 1.2944e+04, 0.0000e+00, ..., 0.0000e+00,
         0.0000e+00, 0.0000e+00],
        [0.0

In [107]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Concatenate

In [76]:
type(X_matrices_train)

numpy.ndarray

In [77]:
X_matrices_train.shape

(114218, 10, 338)

In [79]:
X_pandas_train.shape

(114218, 12)

In [80]:
input_pandas = keras.layers.Input(shape=(12,))
input_matrices = keras.layers.Input(shape=(10, 338))
flattened_matrices = keras.layers.Flatten()(input_matrices)
merged_inputs = keras.layers.concatenate([input_pandas, flattened_matrices])
hidden_layer = keras.layers.Dense(4, activation='sigmoid')(merged_inputs)
output_layer = keras.layers.Dense(1, activation='sigmoid')(hidden_layer)
model = keras.Model(inputs=[input_pandas, input_matrices], outputs=output_layer)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_12 (InputLayer)       [(None, 10, 338)]            0         []                            
                                                                                                  
 input_11 (InputLayer)       [(None, 12)]                 0         []                            
                                                                                                  
 flatten_2 (Flatten)         (None, 3380)                 0         ['input_12[0][0]']            
                                                                                                  
 concatenate_4 (Concatenate  (None, 3392)                 0         ['input_11[0][0]',            
 )                                                                   'flatten_2[0][0]']     

In [81]:
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam',  
    metrics=['accuracy']
)

In [82]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_12 (InputLayer)       [(None, 10, 338)]            0         []                            
                                                                                                  
 input_11 (InputLayer)       [(None, 12)]                 0         []                            
                                                                                                  
 flatten_2 (Flatten)         (None, 3380)                 0         ['input_12[0][0]']            
                                                                                                  
 concatenate_4 (Concatenate  (None, 3392)                 0         ['input_11[0][0]',            
 )                                                                   'flatten_2[0][0]']     

In [83]:
model.fit(
    [X_pandas_train, X_matrices_train],  # Datos de entrada
    y_train,  # Etiquetas
    epochs=10,  # Número de épocas de entrenamiento
    batch_size=32  # Tamaño del lote (batch size)
)

Epoch 1/10
3570/3570 [==============================] - 651s 181ms/step - loss: 0.6944 - accuracy: 0.5029
Epoch 2/10
3570/3570 [==============================] - 618s 173ms/step - loss: 0.6932 - accuracy: 0.5036
Epoch 3/10
3570/3570 [==============================] - 557s 156ms/step - loss: 0.6932 - accuracy: 0.5013
Epoch 4/10
3570/3570 [==============================] - 559s 157ms/step - loss: 0.6932 - accuracy: 0.4998
Epoch 5/10
3570/3570 [==============================] - 559s 157ms/step - loss: 0.6932 - accuracy: 0.5037
Epoch 6/10
3570/3570 [==============================] - 570s 160ms/step - loss: 0.6932 - accuracy: 0.5008
Epoch 7/10
3570/3570 [==============================] - 586s 164ms/step - loss: 0.6932 - accuracy: 0.5007
Epoch 8/10
3570/3570 [==============================] - 564s 158ms/step - loss: 0.6932 - accuracy: 0.5008
Epoch 9/10
3570/3570 [==============================] - 558s 156ms/step - loss: 0.6932 - accuracy: 0.5018
Epoch 10/10
3570/3570 [=======================

In [85]:
model.save('data/transform/model_v1.keras')

In [116]:
from keras.callbacks import EarlyStopping

In [125]:
input_pandas = keras.layers.Input(shape=(12,))
input_matrices = keras.layers.Input(shape=(10, 338))
flattened_matrices = keras.layers.Flatten()(input_matrices)
merged_inputs = keras.layers.concatenate([input_pandas, flattened_matrices])
hidden_layer1 = keras.layers.Dense(32, activation='relu')(merged_inputs)
hidden_layer2 = keras.layers.Dense(16, activation='relu')(hidden_layer1)
hidden_layer3 = keras.layers.Dense(8, activation='relu')(hidden_layer2)
output_layer = keras.layers.Dense(1, activation='sigmoid')(hidden_layer3)

model_v2 = keras.Model(inputs=[input_pandas, input_matrices], outputs=output_layer)

model_v2.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_28 (InputLayer)       [(None, 10, 338)]            0         []                            
                                                                                                  
 input_27 (InputLayer)       [(None, 12)]                 0         []                            
                                                                                                  
 flatten_10 (Flatten)        (None, 3380)                 0         ['input_28[0][0]']            
                                                                                                  
 concatenate_12 (Concatenat  (None, 3392)                 0         ['input_27[0][0]',            
 e)                                                                  'flatten_10[0][0]']   

In [126]:
early_stopping = EarlyStopping(monitor='binary_accuracy', patience=5, restore_best_weights=True)

In [127]:
model_v2.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision()]
)

In [128]:
model_v2.fit(
    [X_pandas_train, X_matrices_train],
    y_train,
    epochs=10,  
    batch_size=32,
    callbacks=[early_stopping]
)

Epoch 1/10
3570/3570 [==============================] - 2098s 586ms/step - loss: 0.6192 - binary_accuracy: 0.8226 - precision_4: 0.8243
Epoch 2/10
3570/3570 [==============================] - 1840s 515ms/step - loss: 0.2000 - binary_accuracy: 0.9232 - precision_4: 0.9233
Epoch 3/10
3570/3570 [==============================] - 1851s 518ms/step - loss: 0.1609 - binary_accuracy: 0.9389 - precision_4: 0.9392
Epoch 4/10
3570/3570 [==============================] - 1861s 521ms/step - loss: 0.1549 - binary_accuracy: 0.9408 - precision_4: 0.9422
Epoch 5/10
3570/3570 [==============================] - 1946s 545ms/step - loss: 0.3654 - binary_accuracy: 0.7650 - precision_4: 0.7128
Epoch 6/10
3570/3570 [==============================] - 2096s 587ms/step - loss: 0.6931 - binary_accuracy: 0.5022 - precision_4: 0.5034
Epoch 7/10
2361/3570 [==================>...........] - ETA: 11:16 - loss: 0.6931 - binary_accuracy: 0.5015 - precision_4: 0.5033

KeyboardInterrupt: 

In [ ]:
model_v2.save('data/transform/model_v2.keras')

In [129]:
input_pandas = keras.layers.Input(shape=(12,))
input_matrices = keras.layers.Input(shape=(10, 338))

dense_layer1 = keras.layers.Dense(16, activation='relu')(input_pandas)
dense_layer2 = keras.layers.Dense(8, activation='relu')(dense_layer1)

conv2d_layer1 = keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu')(tf.expand_dims(input_matrices, axis=-1))
conv2d_layer2 = keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu')(conv2d_layer1)

flattened_matrices = keras.layers.Flatten()(conv2d_layer2)

merged_inputs = keras.layers.concatenate([dense_layer2, flattened_matrices])

hidden_layer1 = keras.layers.Dense(32, activation='relu')(merged_inputs)
hidden_layer2 = keras.layers.Dense(16, activation='relu')(hidden_layer1)
hidden_layer3 = keras.layers.Dense(8, activation='relu')(hidden_layer2)
output_layer = keras.layers.Dense(1, activation='sigmoid')(hidden_layer3)
model_v3 = keras.Model(inputs=[input_pandas, input_matrices], outputs=output_layer)
model_v3.summary()

Model: "model_11"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_30 (InputLayer)       [(None, 10, 338)]            0         []                            
                                                                                                  
 tf.expand_dims (TFOpLambda  (None, 10, 338, 1)           0         ['input_30[0][0]']            
 )                                                                                                
                                                                                                  
 input_29 (InputLayer)       [(None, 12)]                 0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 8, 336, 32)           320       ['tf.expand_dims[0][0]'

In [130]:
model_v3.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.Precision()]
)

In [131]:
early_stopping = EarlyStopping(monitor='binary_accuracy', patience=5, restore_best_weights=True)

In [ ]:
model_v3.fit(
    [X_pandas_train, X_matrices_train],
    y_train,
    epochs=15,
    batch_size=32,
    callbacks=[early_stopping]
)

Epoch 1/15
 556/3570 [===>..........................] - ETA: 4:37:05 - loss: 0.4521 - binary_accuracy: 0.8395 - precision_5: 0.8416

In [ ]:
model_v3.save('data/transform/model_v3.keras')